## Penprapa Wutthijuk StudentID: 6410422023

#### Tried multiple models for initial train and test step. Then selected to apply hyperparameters tuning in Random Forest and SVM due to higher score of accuracy based on the initial train and test result. After applied 2 models (RF and SVM) to data, selected to use SVM as a final model for the test set and submit the prediction of SVM2 and SVM3 as the 2 submissions.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

# 1.Prep Data

In [ ]:
df_train = pd.read_csv('/kaggle/input/dads6003-in-class-competition/train.csv')
print(df_train.isnull().sum())
#print(df_train.shape)

In [ ]:
print(df_train.isnull().sum())
print(df_train.shape)

In [ ]:
#df_train.groupby('y').mean()

df_train.groupby('y').median()

In [ ]:
df_train_median=pd.read_csv('/kaggle/input/dads6003-in-class-competition/train.csv')


In [ ]:
df_train_median['x1'] = df_train_median['x1'].replace(np.NaN ,df_train_median['x1'].median())
df_train_median['x2'] = df_train_median['x2'].replace(np.NaN ,df_train_median['x2'].median())
df_train_median['x3'] = df_train_median['x3'].replace(np.NaN ,df_train_median['x3'].median())
df_train_median['x4'] = df_train_median['x4'].replace(np.NaN ,df_train_median['x4'].median())
df_train_median['x5'] = df_train_median['x5'].replace(np.NaN ,df_train_median['x5'].median())
df_train_median['x6'] = df_train_median['x6'].replace(np.NaN ,df_train_median['x6'].median())
df_train_median['x7'] = df_train_median['x7'].replace(np.NaN ,df_train_median['x7'].median())
df_train_median['x8'] = df_train_median['x8'].replace(np.NaN ,df_train_median['x8'].median())
df_train_median['x10'] = df_train_median['x10'].replace(np.NaN ,df_train_median['x10'].median())
df_train_median['x11'] = df_train_median['x11'].replace(np.NaN ,df_train_median['x11'].median())
df_train_median['x12'] = df_train_median['x12'].replace(np.NaN ,df_train_median['x12'].median())
df_train_median['x13'] = df_train_median['x13'].replace(np.NaN ,df_train_median['x13'].median())
df_train_median['x14'] = df_train_median['x14'].replace(np.NaN ,df_train_median['x14'].median())
df_train_median['x15'] = df_train_median['x15'].replace(np.NaN ,df_train_median['x15'].median())
df_train_median['x16'] = df_train_median['x16'].replace(np.NaN ,df_train_median['x16'].median())
df_train_median['x17'] = df_train_median['x17'].replace(np.NaN ,df_train_median['x17'].median())
df_train_median['x18'] = df_train_median['x18'].replace(np.NaN ,df_train_median['x18'].median())
df_train_median['x19'] = df_train_median['x19'].replace(np.NaN ,df_train_median['x19'].median())
df_train_median['x20'] = df_train_median['x20'].replace(np.NaN ,df_train_median['x20'].median())
print(df_train_median.isnull().sum())

In [ ]:
x_median = df_train_median.iloc[:,0:-1]
y = df_train_median.iloc[:,-1:]

# 2.Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_median, y, test_size=0.25, random_state=42, stratify=y)
print(y_train.shape)
print(X_test.shape)

# 3.Train Model

## 3.1 Random Forest

### 3.1.1 Feature Importance for Random Forest

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
import shap
from matplotlib import pyplot as plt

plt.rcParams.update({'figure.figsize':(12.0,8.0)})
plt.rcParams.update({'font.size':14})


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

In [ ]:
rf.feature_importances_

In [ ]:
rf_feature_names=pd.DataFrame(['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12','x13','x14','x15','x16','x17','x18','x19','x20'])
rf_feature1=pd.DataFrame(rf.feature_importances_)
df_feature_imp1 = pd.concat([rf_feature_names, rf_feature1], axis=1)
df_feature_imp1.columns =['features', 'importance']
df_feature_imp1

In [ ]:
df_feature_imp_sort = df_feature_imp1.sort_values(by = 'importance',ascending = True)
df_feature_imp_sort
plt.barh(df_feature_imp_sort.features, df_feature_imp_sort.importance)

In [ ]:
perm_importance = permutation_importance(rf, X_test, y_test)
#perm_importance

In [ ]:
rf_feature2=pd.DataFrame(perm_importance.importances_mean)
df_feature_imp2 = pd.concat([rf_feature_names, rf_feature2], axis=1)
df_feature_imp2.columns =['features', 'importance']
#df_feature_imp2

In [ ]:
df_feature_imp_sort2 = df_feature_imp2.sort_values(by = 'importance',ascending = True)
df_feature_imp_sort2
plt.barh(df_feature_imp_sort2.features, df_feature_imp_sort2.importance)

In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type='bar')

In [ ]:
#Results from multiple feature importance

score = [('x2',20,20,20,60),
        ('x9',18,19,19,56),
        ('x3',17,18,18,53),
        ('x15',19,16,17,52),
        ('x14',16,17,16,49),
        ('x6',14,15,15,44),
        ('x7',15,14,14,43),
        ('x10',13,13,13,39),
        ('x12',12,11,12,35),
        ('x8',11,12,11,34),
        ('x17',10,10,5,25),
        ('x11',10,6,8,24),
        ('x13',10,4,9,23),
        ('x4',10,2,10,22),
        ('x18',10,8,4,22),
        ('x1',10,9,2,21),
        ('x16',10,5,6,21),
        ('x5',10,7,3,20),
        ('x20',10,1,7,18),
        ('x19',10,3,1,14)]

imfe_result = pd.DataFrame(score, columns = ['features','feat_importance', 'perm_importance','shap','sum_score'])
imfe_result

In [ ]:
X_train2 = X_train.drop(['x19','x20','x5'], axis=1)
X_test2 = X_test.drop(['x19','x20','x5'], axis=1)
X_train2
X_test2

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_rf)
print('Accuracy: %.3f' % (accuracy * 100))

print(r2_score(y_test, y_pred_rf))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train2, y_train)

y_pred_rf2 = rf.predict(X_test2)

accuracy = accuracy_score(y_test, y_pred_rf2)
print('Accuracy: %.3f' % (accuracy * 100))

print(r2_score(y_test, y_pred_rf2))

## 3.1.2 Hyperparameters tuning for Random Forest

In [ ]:
from pprint import pprint

print('Parameters currently in use:\n')
print(rf.get_params())

#### 3.1.2.1 Random Search with Cross Validation

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 150, num = 10)]
max_depth.append(None)

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

In [ ]:
rf2 = RandomForestClassifier(random_state=42)
rf_random = RandomizedSearchCV(estimator=rf2, param_distributions=random_grid,
                              n_iter = 30, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_train2, y_train);

In [ ]:
rf_random.best_params_

rf_random_best_params_
{'n_estimators': 733,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 41,
 'bootstrap': False}

In [ ]:
#rf_random.cv_results_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train2, y_train)

y_pred_base=base_model.predict(X_test2)

accuracy = accuracy_score(y_test, y_pred_base)
print('Accuracy: %.3f' % (accuracy * 100))

print(r2_score(y_test, y_pred_base))

In [ ]:
best_random = rf_random.best_estimator_

y_pred_best=best_random.predict(X_test2)

accuracy = accuracy_score(y_test, y_pred_best)
print('Accuracy: %.3f' % (accuracy * 100))

print(r2_score(y_test, y_pred_best))

#### 3.1.2.2 Grid Search for Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [False],
    'max_depth': [25, 41, 56],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [522, 733, 944]
}

rf2 = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator = rf2, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [ ]:
grid_search.fit(X_train2, y_train);

In [ ]:
grid_search.best_params_

grid_search.best_params_

{'bootstrap': False,
 'max_depth': 25,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 733}

In [ ]:
best_grid = grid_search.best_estimator_

y_pred_grid=best_grid.predict(X_test2)

accuracy = accuracy_score(y_test, y_pred_grid)
print('Accuracy: %.3f' % (accuracy * 100))

print(r2_score(y_test, y_pred_grid))

#### 3.1.2.3 Another Round of Grid Search for Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [False],
    'max_depth': [72, 87, 103],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [1155, 1366]
}

rf2 = RandomForestClassifier(random_state=42)

grid_search_round2 = GridSearchCV(estimator = rf2, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [ ]:
grid_search_round2.fit(X_train2, y_train);

In [ ]:
grid_search_round2.best_params_

grid_search_round2.best_params_
{'bootstrap': False,
 'max_depth': 72,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 1155}

In [ ]:
best_grid_round2 = grid_search_round2.best_estimator_

y_pred_grid2=best_grid_round2.predict(X_test2)

accuracy = accuracy_score(y_test, y_pred_grid2)
print('Accuracy: %.3f' % (accuracy * 100))

print(r2_score(y_test, y_pred_grid2))

### 3.1.3 Final Model for Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier


rfm = RandomForestClassifier(bootstrap=False,
                             ccp_alpha=0.0,
                             class_weight=None,
                             criterion='entropy',
                             max_depth=25,
                             max_features='sqrt',
                             max_leaf_nodes=None,
                             max_samples=None,
                             min_impurity_decrease=0.0,
                             min_samples_leaf=1,
                             min_samples_split=2,
                             min_weight_fraction_leaf=0.0,
                             n_estimators=733,
                             n_jobs=None,
                             oob_score=False,
                             #random_state=123,
                             verbose=0,
                             warm_start=False)
rfm.fit(X_train2, y_train)

y_pred_rfm = rfm.predict(X_test2)

print("Random forest Final")
print(classification_report(y_test, y_pred_rfm))

accuracy = accuracy_score(y_test, y_pred_rfm)
print('Accuracy: %.3f' % (accuracy * 100))

print(r2_score(y_test, y_pred_rfm))

##### Try to improve Final Random Forest Model

In [ ]:
tree_grid = {'n_estimators': [int(x) for x in np.linspace(1, 1500, 50)]}

tree_grid_search = GridSearchCV(rfm, param_grid=tree_grid, verbose = 2, n_jobs=-1, cv = 3,
                                scoring = 'neg_mean_absolute_error')
tree_grid_search.fit(X_train2, y_train);

In [ ]:
tree_grid_search.cv_results_

In [ ]:
tree_grid_search

In [ ]:
def plot_results(model, param = 'n_estimators', name = 'Num Trees'):
   

    test_scores = model.cv_results_['mean_test_score']
    train_time = model.cv_results_['mean_fit_time']
    param_values = list(model.cv_results_[param_name])
    
    plt.subplots(1, 2, figsize=(20, 10))
    plt.subplot(121)

    plt.plot(param_values, test_scores, 'go-', label = 'test')
    plt.ylim(ymin = -0.100, ymax = -0.025)
    plt.legend()
    plt.xlabel(name)
    plt.ylabel('Neg Mean Absolute Error')
    plt.title('Score vs %s' % name)
    
    plt.subplot(122)
    plt.plot(param_values, train_time, 'ro-')
    plt.ylim(ymin = 0.0, ymax = 60.0)
    plt.xlabel(name)
    plt.ylabel('Train Time (sec)')
    plt.title('Training Time vs %s' % name)
    
    
    plt.tight_layout(pad = 4)

In [ ]:
plot_results(tree_grid_search)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier


rfm2 = RandomForestClassifier(bootstrap=False,
                             ccp_alpha=0.0,
                             class_weight=None,
                             criterion='entropy',
                             max_depth=25,
                             max_features='sqrt',
                             max_leaf_nodes=None,
                             max_samples=None,
                             min_impurity_decrease=0.0,
                             min_samples_leaf=1,
                             min_samples_split=2,
                             min_weight_fraction_leaf=0.0,
                             n_estimators=429,
                             n_jobs=None,
                             oob_score=False,
                             verbose=0,
                             warm_start=False)
rfm2.fit(X_train2, y_train)

y_pred_rfm2 = rfm2.predict(X_test2)

print("Random forest Try")
print(classification_report(y_test, y_pred_rfm2))

accuracy = accuracy_score(y_test, y_pred_rfm2)
print('Accuracy: %.3f' % (accuracy * 100))

print(r2_score(y_test, y_pred_rfm2))

## 3.2 SVM

### 3.2.1 Feature Scaling

In [ ]:
cols = X_train.columns

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)

X_test_sc = scaler.transform(X_test)

In [ ]:
X_train_sc = pd.DataFrame(X_train_sc, columns=[cols])
X_test_sc = pd.DataFrame(X_test_sc, columns=[cols])

In [ ]:
X_train_sc.describe()

In [ ]:
#SVM Train Model with default hyperparameters

from sklearn.svm import SVC


from sklearn.metrics import accuracy_score


svc=SVC() 


svc.fit(X_train_sc,y_train)


y_pred=svc.predict(X_test_sc)

print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

### 3.2.2 Hyperparameters Tuning for SVM

#### 3.2.2.1 Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


svc=SVC() 

parameters = [ {'C':[1, 10, 100, 1000], 'kernel':['linear']},
               {'C':[1, 10, 100, 1000], 'kernel':['rbf'], 'gamma':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
               {'C':[1, 10, 100, 1000], 'kernel':['poly'], 'degree': [2,3,4] ,'gamma':[0.01,0.02,0.03,0.04,0.05]} 
              ]

grid_search = GridSearchCV(estimator = svc,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=0)


grid_search.fit(X_train_sc, y_train)

In [ ]:
print('GridSearch CV best score : {:.4f}\n\n'.format(grid_search.best_score_))

print('Parameters that give the best results :','\n\n', (grid_search.best_params_))

print('\n\nEstimator that was chosen by the search :','\n\n', (grid_search.best_estimator_))

Parameters that give the best results : 

 {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}

In [ ]:
print('GridSearch CV score on test set: {0:0.4f}'.format(grid_search.score(X_test_sc, y_test)))

In [ ]:
svc2 = SVC(C=1,kernel='rbf',gamma=0.1)

svc2.fit(X_train_sc, y_train)

y_pred_svc2 = svc2.predict(X_test_sc)
print(y_pred_svc2)

print("SVM")
print(classification_report(y_test, y_pred_svc2))

accuracy = accuracy_score(y_test, y_pred_svc2)
print('Accuracy: %.3f' % (accuracy * 100))

#### 3.2.2.2 Another Grid Search for SVM

In [ ]:
C_range = np.logspace(-1, 1, 3)
print(f'The list of values for C are {C_range}')

gamma_range = np.logspace(-1, 1, 3)
print(f'The list of values for gamma are {gamma_range}')

In [ ]:
param_grid = { 
    "C": C_range,
    "kernel": ['rbf', 'poly'],
    "gamma": gamma_range.tolist()+['scale', 'auto']
    }

scoring = ['accuracy']
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [ ]:
grid_search_2 = GridSearchCV(estimator=svc, 
                           param_grid=param_grid, 
                           scoring=scoring, 
                           refit='accuracy', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)

grid_result = grid_search_2.fit(X_train_sc, y_train)
grid_result

In [ ]:
print('GridSearch CV best score : {:.4f}\n\n'.format(grid_search_2.best_score_))

print('Parameters that give the best results :','\n\n', (grid_search_2.best_params_))

print('\n\nEstimator that was chosen by the search :','\n\n', (grid_search_2.best_estimator_))

#### 3.2.2.3 Grid Search using Bayesian Opti

In [ ]:
# Space
space = {
    'C' : hp.choice('C', C_range),
    'gamma' : hp.choice('gamma', gamma_range.tolist()+['scale', 'auto']),
    'kernel' : hp.choice('kernel', ['rbf', 'poly'])
}
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
# Objective function
def objective(params):
    
    svc = SVC(**params)
    scores = cross_val_score(svc, X_train_sc, y_train, cv=kfold, scoring='accuracy', n_jobs=-1)
    # Extract the best score
    best_score = max(scores)
    # Loss must be minimized
    loss = - best_score
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}
# Trials to track progress
bayes_trials = Trials()
# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 1000, trials = bayes_trials)

In [ ]:
print(best) #index of the best parameters

print(space_eval(space, best)) #values of best para

Bayesian's result: the values of the best parameters are {'C': 1.0, 'gamma': 'scale', 'kernel': 'rbf'}

In [ ]:
svc3 = SVC(C=1,kernel='rbf',gamma='scale')

svc3.fit(X_train_sc, y_train)

y_pred_svc3 = svc3.predict(X_test_sc)
print(y_pred_svc3)

print("SVM")
print(classification_report(y_test, y_pred_svc3))

accuracy = accuracy_score(y_test, y_pred_svc3)
print('Accuracy: %.3f' % (accuracy * 100))

#### 3.2.2.4 Tuning using Random Search

In [ ]:
C_range = np.logspace(-10, 10, 21)
print(f'The list of values for C are {C_range}')

gamma_range = np.logspace(-10, 10, 21)
print(f'The list of values for gamma are {gamma_range}')

In [ ]:
param_grid = { 
    "C": C_range,
    "kernel": ['rbf', 'poly'],
    "gamma": gamma_range.tolist()+['scale', 'auto']
    }

scoring = ['accuracy']
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

random_search = RandomizedSearchCV(estimator=svc, 
                           param_distributions=param_grid, 
                           n_iter=100,
                           scoring=scoring, 
                           refit='accuracy', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)

random_result = random_search.fit(X_train_sc, y_train)
random_result

In [ ]:
print(f'The best accuracy score for the training dataset is {random_result.best_score_:.4f}')

print(f'The best hyperparameters are {random_result.best_params_}')

print(f'The accuracy score for the testing dataset is {random_search.score(X_test_sc, y_test):.4f}')

The best hyperparameters from random search are {'kernel': 'rbf', 'gamma': 0.01, 'C': 10.0}

In [ ]:
svc4 = SVC(C=10,kernel='rbf',gamma=0.01)

svc4.fit(X_train_sc, y_train)

y_pred_svc4 = svc4.predict(X_test_sc)
print(y_pred_svc4)

print("SVM")
print(classification_report(y_test, y_pred_svc4))

accuracy = accuracy_score(y_test, y_pred_svc4)
print('Accuracy: %.3f' % (accuracy * 100))

# 4.Test Model

## 4.1 Test Model for SVM

In [ ]:
df_test = pd.read_csv('/kaggle/input/dads6003-in-class-competition/test.csv')
#print(df_test.isnull().sum())
Xnew = df_test.iloc[:,:]
Xnew

In [ ]:
cols = Xnew.columns

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_test_new = scaler.fit_transform(Xnew)
X_test_new2 = scaler.transform(Xnew)

In [ ]:
X_test_new = pd.DataFrame(X_test_new, columns=[cols])
X_test_new2 = pd.DataFrame(X_test_new2, columns=[cols])

In [ ]:
X_test_new.head()

In [ ]:
X_test_new2.head()

In [ ]:
#svc2 = SVC(C=1,kernel='rbf',gamma=0.1)

Ynew_sc = svc2.predict(X_test_new)
Ynew_sc

In [ ]:
#svc3 = SVC(C=1,kernel='rbf',gamma='scale')

Ynew_sc2 = svc3.predict(X_test_new)
Ynew_sc2

In [ ]:
df_test['y']=Ynew_sc
submission = df_test[['y']]
submission.rename(columns={'y':'Expected'},inplace=True)
submission.index = np.arange(1,len(submission)+1)
submission.index.name='Id'
submission.head()
submission.to_csv('20th_submission.csv', index=True)

In [ ]:
df_test['y']=Ynew_sc2
submission = df_test[['y']]
submission.rename(columns={'y':'Expected'},inplace=True)
submission.index = np.arange(1,len(submission)+1)
submission.index.name='Id'
submission.head()
submission.to_csv('21st_submission.csv', index=True)

## 4.2 Test Model for Random Forest

In [ ]:
df_test = pd.read_csv('/kaggle/input/dads6003-in-class-competition/test.csv')
#print(df_test.isnull().sum())
Xnew = df_test.iloc[:,:]
Xnew

In [ ]:
Xnew_2 = Xnew.drop(['x19','x20','x5'], axis=1)
Xnew_2

In [ ]:
Ynew = rfm.predict(Xnew_2)
Ynew

In [ ]:
df_test['y']=Ynew
submission = df_test[['y']]
submission.rename(columns={'y':'Expected'},inplace=True)
submission.index = np.arange(1,len(submission)+1)
submission.index.name='Id'
submission.head()
submission.to_csv('16th_submission.csv', index=True)

## Explored the data and tried multiple algorithms before the final model

In [ ]:
#df_train = pd.read_csv('/kaggle/input/dads6003-in-class-competition/train.csv')
#df_train.head()
#print(df_train.isnull().sum())

In [ ]:
#df_train_mean=pd.read_csv('/kaggle/input/dads6003-in-class-competition/train.csv')
#df_train_mean['x1'] = df_train_mean['x1'].replace(np.NaN ,df_train_mean['x1'].mean())
#df_train_mean['x2'] = df_train_mean['x2'].replace(np.NaN ,df_train_mean['x2'].mean())
#df_train_mean['x3'] = df_train_mean['x3'].replace(np.NaN ,df_train_mean['x3'].mean())
#df_train_mean['x4'] = df_train_mean['x4'].replace(np.NaN ,df_train_mean['x4'].mean())
#df_train_mean['x5'] = df_train_mean['x5'].replace(np.NaN ,df_train_mean['x5'].mean())
#df_train_mean['x6'] = df_train_mean['x6'].replace(np.NaN ,df_train_mean['x6'].mean())
#df_train_mean['x7'] = df_train_mean['x7'].replace(np.NaN ,df_train_mean['x7'].mean())
#df_train_mean['x8'] = df_train_mean['x8'].replace(np.NaN ,df_train_mean['x8'].mean())
#df_train_mean['x10'] = df_train_mean['x10'].replace(np.NaN ,df_train_mean['x10'].mean())
#df_train_mean['x11'] = df_train_mean['x11'].replace(np.NaN ,df_train_mean['x11'].mean())
#df_train_mean['x12'] = df_train_mean['x12'].replace(np.NaN ,df_train_mean['x12'].mean())
#df_train_mean['x13'] = df_train_mean['x13'].replace(np.NaN ,df_train_mean['x13'].mean())
#df_train_mean['x14'] = df_train_mean['x14'].replace(np.NaN ,df_train_mean['x14'].mean())
#df_train_mean['x15'] = df_train_mean['x15'].replace(np.NaN ,df_train_mean['x15'].mean())
#df_train_mean['x16'] = df_train_mean['x16'].replace(np.NaN ,df_train_mean['x16'].mean())
#df_train_mean['x17'] = df_train_mean['x17'].replace(np.NaN ,df_train_mean['x17'].mean())
#df_train_mean['x18'] = df_train_mean['x18'].replace(np.NaN ,df_train_mean['x18'].mean())
#df_train_mean['x19'] = df_train_mean['x19'].replace(np.NaN ,df_train_mean['x19'].mean())
#df_train_mean['x20'] = df_train_mean['x20'].replace(np.NaN ,df_train_mean['x20'].mean())
#print(df_train_mean['x1'])

In [ ]:
#Linear Regression

#from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import mean_squared_error
#lin_reg = LinearRegression()

#lin_reg.fit(x_median, y)
#print(lin_reg.coef_)
#print(lin_reg.intercept_)
#print("R^2 ", lin_reg.score(x_median,y))

#y_pred_lin = lin_reg.predict(x_median)
#print("MSE Linear = ", mean_squared_error(y,y_pred_lin))

#plt.plot(x_median,y,'o')
#plt.plot(x_median,y_pred_lin,'r-',label="linear regression")
#plt.legend() 
#plt.show()

In [ ]:
#Logistic Regression

#from sklearn.metrics import accuracy_score
#from sklearn.neighbors import KNeighborsClassifier
#X_train, X_test, y_train, y_test = train_test_split(x_median, y, test_size=0.2, random_state=123, stratify=y)
#print(y_train.shape)
#print(X_test.shape)
#from sklearn.linear_model import LogisticRegression
#from sklearn import metrics
#logreg = LogisticRegression()
#logreg.fit(X_train, y_train)

#y_pred_lg = logreg.predict(X_test)
#print(y_pred_lg)

#accuracy = accuracy_score(y_test, y_pred_lg)
#print('Accuracy: %.3f' % (accuracy * 100))

In [ ]:
#Decision Tree

#from sklearn.metrics import accuracy_score
#from sklearn import tree
#X_train, X_test, y_train, y_test = train_test_split(x_median, y, test_size=0.2, random_state=123, stratify=y)

#dct = tree.DecisionTreeClassifier()
#dct.fit(X_train, y_train)

#y_pred_dct = dct.predict(X_test)
#print(y_pred_dct)

#print("Decision tree")
#print(classification_report(y_test, y_pred_dct))

#accuracy = accuracy_score(y_test, y_pred_dct)
#print('Accuracy: %.3f' % (accuracy * 100))

In [ ]:
#Random Forest

#from sklearn.metrics import accuracy_score
#from sklearn.metrics import r2_score
#from sklearn.ensemble import RandomForestClassifier
#X_train, X_test, y_train, y_test = train_test_split(x_median, y, test_size=0.2, random_state=123, stratify=y)

#rfm0 = RandomForestClassifier(n_estimators=90,oob_score=True, n_jobs=-1,max_features=None, min_samples_leaf=2,random_state=101)
#rfm0.fit(X_train, y_train)

#y_pred_rfm0 = rfm0.predict(X_test)
#print(y_pred_rfm0)

#print("Random forest")
#print(classification_report(y_test, y_pred_rfm0))

#accuracy = accuracy_score(y_test, y_pred_rfm0)
#print('Accuracy: %.3f' % (accuracy * 100))

#print(r2_score(y_test, y_pred_rfm0))

In [ ]:
#kNN


#from sklearn.metrics import accuracy_score
#from sklearn.neighbors import KNeighborsClassifier
#X_train, X_test, y_train, y_test = train_test_split(x_median, y, test_size=0.2, random_state=123, stratify=y)

#model_kNN = KNeighborsClassifier()
#model_kNN.fit(X_train, y_train)

#y_pred_knn = model_kNN.predict(X_test)

#accuracy = accuracy_score(y_test, y_pred_knn)
#print('Accuracy: %.3f' % (accuracy * 100))

In [ ]:
#Naive Bayes

#from sklearn.metrics import accuracy_score
#from sklearn.naive_bayes import GaussianNB
#X_train, X_test, y_train, y_test = train_test_split(x_median, y, test_size=0.2, random_state=123, stratify=y)

#nb = GaussianNB()
#nb.fit(X_train, y_train)

#y_pred_nb = nb.predict(X_test)
#print(y_pred_nb)

#print("Naive Bayes")
#print(classification_report(y_test, y_pred_nb))

#accuracy = accuracy_score(y_test, y_pred_nb)
#print('Accuracy: %.3f' % (accuracy * 100))

In [ ]:
#SVM

#from sklearn.metrics import accuracy_score
#from sklearn.svm import SVC
#X_train, X_test, y_train, y_test = train_test_split(x_median, y, test_size=0.2, random_state=123, stratify=y)

#svm = SVC(kernel='linear', C=0.025, random_state=101)
#svm.fit(X_train, y_train)

#y_pred_svm = svm.predict(X_test)
#print(y_pred_svm)

#print("SVM")
#print(classification_report(y_test, y_pred_svm))

#accuracy = accuracy_score(y_test, y_pred_svm)
#print('Accuracy: %.3f' % (accuracy * 100))

In [ ]:
#Stochastic Gradient Descent

#from sklearn.metrics import accuracy_score
#from sklearn.linear_model import SGDClassifier
#X_train, X_test, y_train, y_test = train_test_split(x_median, y, test_size=0.2, random_state=123, stratify=y)

#sgd = SGDClassifier(loss='modified_huber', shuffle=True, random_state=101)
#sgd.fit(X_train, y_train)

#y_pred_sgd = sgd.predict(X_test)
#print(y_pred_sgd)

#print("Stochastic Gradient Descent")
#print(classification_report(y_test, y_pred_sgd))

#accuracy = accuracy_score(y_test, y_pred_sgd)
#print('Accuracy: %.3f' % (accuracy * 100))

In [ ]:
#K-Mean

#from sklearn.metrics import accuracy_score
#from sklearn.cluster import KMeans
#X_train, X_test, y_train, y_test = train_test_split(x_median, y, test_size=0.2, random_state=123, stratify=y)

#nc=2
#kmeans = KMeans(n_clusters=nc, random_state=0)
#kmeans.fit(X_train, y_train)

#y_pred_km = kmeans.predict(X_test)
#print(y_pred_km)

#print("K-Mean")
#print(classification_report(y_test, y_pred_km))

#accuracy = accuracy_score(y_test, y_pred_km)
#print('Accuracy: %.3f' % (accuracy * 100))

## Apply SFS

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X_train_sfs = scaler.fit_transform(X_train)
X_test_sfs = scaler.transform(X_test)

y_train_sfs = np.asarray(y_train)
y_test_sfs = np.asarray(y_test)

lr = LinearRegression()

sfs1 = SFS(lr,
          k_features=20,
          forward=True,
          floating=False,
          scoring='neg_mean_squared_error',
          cv=10)

sfs1 = sfs1.fit(X_train_sfs, y_train_sfs)
fig = plot_sfs(sfs1.get_metric_dict(), kind='std_err')

plt.title('Sequential Forward Selection with Standard Error')
plt.grid()
plt.show()